# PRE-PROCESS

In [40]:
import pandas as pd
import sqlite3
import re


In [41]:
csv_path = "V6 Updated_In-Depth_Analysis_of_Mental_Health_Dataset.csv"
df = pd.read_csv(csv_path)

df.head()
df.dtypes


gender                      object
age                          int64
current_fitness_lvl         object
exercise_timeline_weekly     int64
workout_companion           object
primary_goal                object
biggest_mental_shift        object
scale_feelings               int64
health_improvement          object
impact_on_daylife           object
diet_sleep_impact           object
fitness_tracker             object
sleeptime                   object
follow_dietroutine          object
hydration_attention         object
favorite_workout            object
post_impacts                object
music_impact                object
recovery_methods            object
additional_impacts          object
family_reaction             object
biggest_challenge           object
dtype: object

# Create a SQLite database from the CSV

In [42]:
# Create (or connect to) a SQLite database file
conn = sqlite3.connect("mental_health.db")

# Write the DataFrame into a SQL table
table_name = "mental_health"
df.to_sql(table_name, conn, if_exists="replace", index=False)

# Quick check: how many rows?
cur = conn.cursor()
cur.execute(f"SELECT COUNT(*) FROM {table_name}")
print("Rows in table:", cur.fetchone()[0])


Rows in table: 1930


# Utility function to run SQL and get a DataFrame

In [43]:
def run_sql(query: str):
    """
    Run a SQL query on the mental_health database and return a pandas DataFrame.
    """
    print("Executing SQL:\n", query)
    return pd.read_sql_query(query, conn)


In [44]:
test_query = f"""
SELECT primary_goal, AVG(age) AS avg_age, COUNT(*) AS n
FROM {table_name}
GROUP BY primary_goal
ORDER BY n DESC;
"""

run_sql(test_query)


Executing SQL:
 
SELECT primary_goal, AVG(age) AS avg_age, COUNT(*) AS n
FROM mental_health
GROUP BY primary_goal
ORDER BY n DESC;



,primary_goal,avg_age,n
0,Weight loss,25.996599,294
1,Muscle gain,26.468310,284
2,Mental stress relief,25.406699,209
3,Physical health,25.445545,202
4,"Physical health, Mental stress relief",27.935252,139
5,"Mental stress relief, Muscle gain",28.481818,110
6,"Physical health, Muscle gain",27.161905,105
7,"Physical health, Mental stress relief, Muscle ...",23.819048,105
8,"Physical health, Mental stress relief, Weight ...",24.772727,88
9,"Weight loss, Muscle gain",28.205128,78


# Describe the schema for the LLM

In [45]:
SCHEMA_DESCRIPTION = f"""
You are working with a SQLite database that has ONE table: {table_name}.

Table {table_name} has the following columns:

- gender (TEXT)
- age (INTEGER)
- current_fitness_lvl (TEXT)
- exercise_timeline_weekly (INTEGER) -- number of workout sessions/week
- workout_companion (TEXT)
- primary_goal (TEXT) -- e.g., Weight loss, Muscle gain, Mental health, etc.
- biggest_mental_shift (TEXT)
- scale_feelings (INTEGER) -- numeric rating scale
- health_improvement (TEXT)
- impact_on_daylife (TEXT)
- diet_sleep_impact (TEXT)
- fitness_tracker (TEXT)
- sleeptime (TEXT)
- follow_dietroutine (TEXT)
- hydration_attention (TEXT)
- favorite_workout (TEXT)
- post_impacts (TEXT)
- music_impact (TEXT)
- recovery_methods (TEXT)
- additional_impacts (TEXT)
- family_reaction (TEXT)
- biggest_challenge (TEXT)
"""
print(SCHEMA_DESCRIPTION)



You are working with a SQLite database that has ONE table: mental_health.

Table mental_health has the following columns:

- gender (TEXT)
- age (INTEGER)
- current_fitness_lvl (TEXT)
- exercise_timeline_weekly (INTEGER) -- number of workout sessions/week
- workout_companion (TEXT)
- primary_goal (TEXT) -- e.g., Weight loss, Muscle gain, Mental health, etc.
- biggest_mental_shift (TEXT)
- scale_feelings (INTEGER) -- numeric rating scale
- health_improvement (TEXT)
- impact_on_daylife (TEXT)
- diet_sleep_impact (TEXT)
- fitness_tracker (TEXT)
- sleeptime (TEXT)
- follow_dietroutine (TEXT)
- hydration_attention (TEXT)
- favorite_workout (TEXT)
- post_impacts (TEXT)
- music_impact (TEXT)
- recovery_methods (TEXT)
- additional_impacts (TEXT)
- family_reaction (TEXT)
- biggest_challenge (TEXT)



# Design the Text2SQL prompt

In [46]:
TEXT2SQL_SYSTEM_PROMPT = """
You are an expert data analyst who writes SQL for a SQLite database.

RULES:
- Write ONLY a SQL SELECT query, no explanation.
- Never use DELETE, INSERT, UPDATE, DROP, or CREATE.
- Use the table name exactly as given.
- Use valid SQLite syntax.
"""

TEXT2SQL_USER_TEMPLATE = """
Here is the database schema:
{schema}

User question:
{question}

Write a single SQL SELECT query that answers the question.
Return ONLY the SQL, no backticks, no explanation.
"""


# LLM call (stub + example)

In [47]:
# Stub (model-agnostic)

In [52]:
def call_llm(prompt: str) -> str:
    """
    Stub for calling an LLM. 
    Replace the body with the API or local model you want to use.
    It must return a plain text string with the model's response.
    """
    raise NotImplementedError("Implement this with your chosen LLM API or local model.")



In [63]:
import os

# Tell Hugging Face to cache everything on E:
os.environ["HF_HOME"] = r"E:\hf_models"
os.environ["TRANSFORMERS_CACHE"] = r"E:\hf_models\transformers"
os.environ["HF_DATASETS_CACHE"] = r"E:\hf_models\datasets"  # just in case you use datasets later


In [64]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Choose a small instruct/chat model
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
CACHE_DIR = r"E:\hf_models\transformers"   # same folder as above

# Load tokenizer and model (they will be downloaded to E: now)
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    cache_dir=CACHE_DIR
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    cache_dir=CACHE_DIR,
    device_map="auto",          # GPU if available, else CPU
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

# Build a text-generation pipeline
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,          # enough for SQL
    do_sample=False,             # deterministic (good for SQL)
)


C:\Users\User\anaconda3\envs\llm_env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\hf_models\transformers\models--microsoft--Phi-3-mini-4k-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.69s/it]
Device set to use cuda:0


# Connect the model to your Text2SQL pipeline

In [66]:
def call_llm(prompt: str) -> str:
    """
    Use the local Phi-3 model (via Hugging Face) to generate SQL.
    """

    # Combine system + user prompt
    full_prompt = TEXT2SQL_SYSTEM_PROMPT.strip() + "\n\n" + prompt.strip()

    # Generate completion
    outputs = text_gen(
        full_prompt,
        num_return_sequences=1
    )

    # Pipeline returns: [ { "generated_text": "<prompt> + <completion>" } ]
    full_text = outputs[0]["generated_text"]

    # Keep only the part AFTER the prompt
    completion = full_text[len(full_prompt):].strip()

    return completion


In [ ]:
#HELPER

In [69]:
import re

def extract_sql_from_response(raw_response: str) -> str:
    """
    Given the raw LLM response, extract the first SQL SELECT statement.
    Handles cases where the model adds headers, explanations, or extra text.
    """
    text = raw_response.strip()

    # Remove common code-fence markers if present
    text = text.replace("```sql", "").replace("```", "").strip()

    # Find the first occurrence of 'select' (case-insensitive)
    match = re.search(r"\bselect\b", text, re.IGNORECASE)
    if not match:
        raise ValueError(f"No SELECT statement found in LLM response:\n{text}")

    # Take everything from 'select' onward
    sql = text[match.start():]

    # Optionally cut at the first semicolon (end of statement)
    semicolon_idx = sql.find(";")
    if semicolon_idx != -1:
        sql = sql[:semicolon_idx + 1]

    return sql.strip()


In [70]:
def question_to_sql(question: str) -> str:
    """
    Given a natural language question, ask the LLM to generate a SQL query.
    """
    user_prompt = TEXT2SQL_USER_TEMPLATE.format(
        schema=SCHEMA_DESCRIPTION,
        question=question
    )

    full_prompt = user_prompt  # system prompt is added inside call_llm

    raw_response = call_llm(full_prompt)

    # Use the robust extractor instead of naive .startswith("select")
    sql = extract_sql_from_response(raw_response)

    # Final safety check
    if not sql.lower().startswith("select"):
        raise ValueError(f"Extracted SQL does not start with SELECT:\n{sql}")

    return sql


In [71]:
q1 = "How many total respondents are there in the dataset?"
df_ans1 = ask_text2sql(q1)
df_ans1



Natural language question:
 How many total respondents are there in the dataset?

Generated SQL:
 SELECT COUNT(*) FROM mental_health;
Executing SQL:
 SELECT COUNT(*) FROM mental_health;


,COUNT(*)
0,1930


In [72]:
questions = [
    "How many total respondents are there in the dataset?",
    "What is the average age of all respondents?",
    "How many respondents have their primary goal as weight loss?",
    "For each primary_goal, show the number of respondents.",
    "For each primary_goal, show the average age and the number of respondents.",
    "For each current_fitness_lvl, what is the average exercise_timeline_weekly?",
    "How many respondents use a fitness tracker?",
    "How many respondents use a fitness tracker and also follow a diet routine?",
    "For each gender, show the average scale_feelings score and the number of respondents.",
    "For each favorite_workout, show the number of respondents, ordered by count descending.",
    "Among respondents who exercise at least 4 times per week, what is the average scale_feelings score?",
    "Which primary_goal has the highest average scale_feelings score?",
    "For each hydration_attention category, what is the average scale_feelings score?",
    "For each current_fitness_lvl, show the average age and average exercise_timeline_weekly, ordered by average exercise_timeline_weekly descending.",
    "How many respondents report that their diet_sleep_impact is 'Very Positive'?"
]

for i, q in enumerate(questions, start=1):
    print("\n" + "="*60)
    print(f"Q{i}: {q}")
    try:
        df_ans = ask_text2sql(q)
        display(df_ans.head(20))
    except Exception as e:
        print("Error:", e)



Q1: How many total respondents are there in the dataset?

Natural language question:
 How many total respondents are there in the dataset?

Generated SQL:
 SELECT COUNT(*) FROM mental_health;
Executing SQL:
 SELECT COUNT(*) FROM mental_health;


,COUNT(*)
0,1930



Q2: What is the average age of all respondents?

Natural language question:
 What is the average age of all respondents?

Generated SQL:
 SELECT AVG(age) FROM mental_health;
Executing SQL:
 SELECT AVG(age) FROM mental_health;


,AVG(age)
0,26.419689



Q3: How many respondents have their primary goal as weight loss?

Natural language question:
 How many respondents have their primary goal as weight loss?

Generated SQL:
 SELECT COUNT(*) FROM mental_health WHERE primary_goal = 'Weight loss';
Executing SQL:
 SELECT COUNT(*) FROM mental_health WHERE primary_goal = 'Weight loss';


...

,hydration_attention,average_scale_feelings
0,None at all,5.612245
1,Quite a lot,6.757542
2,Very little,5.539007
3,Very much,7.817694



Q14: For each current_fitness_lvl, show the average age and average exercise_timeline_weekly, ordered by average exercise_timeline_weekly descending.

Natural language question:
 For each current_fitness_lvl, show the average age and average exercise_timeline_weekly, ordered by average exercise_timeline_weekly descending.

Generated SQL:
 SELECT current_fitness_lvl, AVG(age) AS average_age, AVG(exercise_timeline_weekly) AS average_exercise_timeline_weekly
FROM mental_health
GROUP BY current_fitness_lvl
ORDER BY average_exercise_timeline_weekly DESC;
Executing SQL:
 SELECT current_fitness_lvl, AVG(age) AS average_age, AVG(exercise_timeline_weekly) AS average_exercise_timeline_weekly
FROM mental_health
GROUP BY current_fitness_lvl
ORDER BY average_exercise_timeline_weekly DESC;


,current_fitness_lvl,average_age,average_exercise_timeline_weekly
0,Expert,30.635838,5.832370
1,Advanced,29.069853,5.132353
2,Intermediate,25.841787,3.820048
3,Beginner,24.940639,2.697108



Q15: How many respondents report that their diet_sleep_impact is 'Very Positive'?

Natural language question:
 How many respondents report that their diet_sleep_impact is 'Very Positive'?

Generated SQL:
 SELECT COUNT(*) FROM mental_health WHERE diet_sleep_impact = 'Very Positive';
Executing SQL:
 SELECT COUNT(*) FROM mental_health WHERE diet_sleep_impact = 'Very Positive';


,COUNT(*)
0,0


In [73]:
mixed_questions = [
    # 1. Descriptive + counts
    "List all distinct primary_goal values and show how many respondents fall into each goal, ordered from most common to least common.",
    "For each current_fitness_lvl, show the number of respondents and the average age.",
    "For each workout_companion category, show how many respondents are in that category and their average exercise_timeline_weekly.",
    "Show the top 10 most frequent biggest_challenge values and how many respondents reported each one.",
    "For each hydration_attention category, show the number of respondents and their average scale_feelings score.",

    # 2. Behavior + mental state (mixed)
    "For each primary_goal, show the average scale_feelings score and the number of respondents.",
    "Among respondents who use a fitness tracker, what is the average exercise_timeline_weekly and average scale_feelings score?",
    "For each favorite_workout, show the average scale_feelings score and the number of respondents, ordered by average scale_feelings descending.",
    "For each diet_sleep_impact category, show the number of respondents and their average scale_feelings score.",
    "Among respondents whose impact_on_daylife is very positive, what is their average exercise_timeline_weekly and scale_feelings score?",

    # 3. Comparisons between groups
    "Compare the average age and average exercise_timeline_weekly for respondents whose primary_goal is weight loss versus those whose primary_goal is muscle gain.",
    "Compare the average scale_feelings score for respondents who follow a diet routine versus those who do not follow a diet routine.",
    "Compare the average exercise_timeline_weekly for respondents who work out alone versus those who work out with friends.",
    "For each gender, show the average scale_feelings score and average exercise_timeline_weekly, ordered by average scale_feelings descending.",
    "Among respondents who use a fitness tracker, compare the average scale_feelings score for different primary_goal categories.",

    # 4. Filtered subset + grouped summary
    "Among respondents whose primary_goal is mental health, show the number of respondents for each current_fitness_lvl and the average scale_feelings score within each level.",
    "Among respondents who exercise at least 4 times per week, show the number of respondents for each primary_goal and the average scale_feelings score.",
    "For respondents who report that their family_reaction is very supportive, show the number of respondents for each primary_goal and the average scale_feelings score.",
    "Among respondents who follow a diet routine, show the number of respondents for each favorite_workout and the average scale_feelings score.",
    "Among respondents who do not use a fitness tracker, show the number of respondents for each current_fitness_lvl and the average exercise_timeline_weekly.",

    # 5. “Show me examples” / textual focus
    "Show 10 example rows with columns: age, primary_goal, workout_companion, favorite_workout, scale_feelings.",
    "Show 10 example biggest_challenge responses along with age and primary_goal.",
    "Show 10 example rows where the primary_goal is mental health, including columns: age, current_fitness_lvl, exercise_timeline_weekly, biggest_mental_shift.",
    "Show 10 example rows where the family_reaction is negative, including columns: age, primary_goal, impact_on_daylife, biggest_challenge.",
    "Show 10 example rows where hydration_attention is high, including columns: age, primary_goal, favorite_workout, scale_feelings.",

    # 6. Ranking / “best/worst” questions
    "Which primary_goal has the highest average exercise_timeline_weekly, and what is that average along with the number of respondents?",
    "Which favorite_workout has the highest average scale_feelings score, and how many respondents chose it?",
    "Which current_fitness_lvl has the highest average scale_feelings score, and how many respondents are in that level?",
    "Which hydration_attention category has the highest average scale_feelings score, and how many respondents are in that category?",
    "Which biggest_challenge appears most frequently in the dataset, and how many respondents reported it?"
]


In [74]:
for i, q in enumerate(mixed_questions, start=1):
    print("\n" + "="*80)
    print(f"Q{i}: {q}\n")
    try:
        df_ans = ask_text2sql(q)
        display(df_ans.head(20))   # show first 20 rows if big
    except Exception as e:
        print("Error:", e)



Q1: List all distinct primary_goal values and show how many respondents fall into each goal, ordered from most common to least common.


Natural language question:
 List all distinct primary_goal values and show how many respondents fall into each goal, ordered from most common to least common.

Generated SQL:
 SELECT primary_goal, COUNT(*) AS respondent_count
FROM mental_health
GROUP BY primary_goal
ORDER BY respondent_count DESC;
Executing SQL:
 SELECT primary_goal, COUNT(*) AS respondent_count
FROM mental_health
GROUP BY primary_goal
ORDER BY respondent_count DESC;


,primary_goal,respondent_count
0,Weight loss,294
1,Muscle gain,284
2,Mental stress relief,209
3,Physical health,202
4,"Physical health, Mental stress relief",139
5,"Mental stress relief, Muscle gain",110
6,"Physical health, Muscle gain",105
7,"Physical health, Mental stress relief, Muscle ...",105
8,"Physical health, Mental stress relief, Weight ...",88
9,"Weight loss, Muscle gain",78



Q2: For each current_fitness_lvl, show the number of respondents and the average age.


Natural language question:
 For each current_fitness_lvl, show the number of respondents and the average age.

Generated SQL:
 SELECT current_fitness_lvl, COUNT(*) AS number_of_respondents, AVG(age) AS average_age
FROM mental_health
GROUP BY current_fitness_lvl;
Executing SQL:
 SELECT current_fitness_lvl, COUNT(*) AS number_of_respondents, AVG(age) AS average_age
FROM mental_health
GROUP BY current_fitness_lvl;


,current_fitness_lvl,number_of_respondents,average_age
0,Advanced,272,29.069853
1,Beginner,657,24.940639
2,Expert,173,30.635838
3,Intermediate,828,25.841787



Q3: For each workout_companion category, show how many respondents are in that category and their average exercise_timeline_weekly.


Natural language question:
 For each workout_companion category, show how many respondents are in that category and their average exercise_timeline_weekly.

Generated SQL:
 SELECT workout_companion, COUNT(*) AS respondent_count, AVG(exercise_timeline_weekly) AS average_exercise_timeline_weekly
FROM mental_health
GROUP BY workout_companion;
Executing SQL:
 SELECT workout_companion, COUNT(*) AS respondent_count, AVG(exercise_timeline_weekly) AS average_exercise_timeline_weekly
FROM mental_health
GROUP BY workout_companion;


...

,current_fitness_lvl,average_scale_feelings,respondent_count
0,Expert,8.462428,173



Q29: Which hydration_attention category has the highest average scale_feelings score, and how many respondents are in that category?


Natural language question:
 Which hydration_attention category has the highest average scale_feelings score, and how many respondents are in that category?

Generated SQL:
 SELECT hydration_attention, AVG(scale_feelings) AS average_scale_feelings, COUNT(*) AS respondent_count
FROM mental_health
GROUP BY hydration_attention
ORDER BY average_scale_feelings DESC
LIMIT 1;
Executing SQL:
 SELECT hydration_attention, AVG(scale_feelings) AS average_scale_feelings, COUNT(*) AS respondent_count
FROM mental_health
GROUP BY hydration_attention
ORDER BY average_scale_feelings DESC
LIMIT 1;


,hydration_attention,average_scale_feelings,respondent_count
0,Very much,7.817694,373



Q30: Which biggest_challenge appears most frequently in the dataset, and how many respondents reported it?


Natural language question:
 Which biggest_challenge appears most frequently in the dataset, and how many respondents reported it?

Generated SQL:
 SELECT biggest_challenge, COUNT(*) AS frequency
FROM mental_health
GROUP BY biggest_challenge
ORDER BY frequency DESC
LIMIT 1;
Executing SQL:
 SELECT biggest_challenge, COUNT(*) AS frequency
FROM mental_health
GROUP BY biggest_challenge
ORDER BY frequency DESC
LIMIT 1;


,biggest_challenge,frequency
0,Lack of time,427
